In [1]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense , Flatten
from tensorflow.keras.applications import VGG16

In [2]:
train_dir = './dataset/train/'
test_dir = './dataset/test/'
val_dir = './dataset/val/'

In [3]:
generator = ImageDataGenerator(rescale = 1./255)
train_generator = generator.flow_from_directory(
    directory = train_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
test_generator = generator.flow_from_directory(
    directory = test_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
val_generator = generator.flow_from_directory(
    directory = val_dir, 
    target_size = (150,150), 
    batch_size = 100, 
    class_mode = 'categorical'
)
print(train_generator.class_indices)

Found 320 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
Found 40 images belonging to 4 classes.
{'90leg': 0, 'head up': 1, 'lay down': 2, 'lay up': 3}


In [4]:
# 전이학습 -> VGG모델 사용
conv_base = VGG16(
    weights = 'imagenet',
    include_top = False,
    input_shape = (150,150,3)
)

In [5]:
# 가중치 동결 
conv_base.trainable=False

In [6]:
model = Sequential()

model.add(conv_base)
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(4, activation='softmax'))

In [7]:
model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics = ['accuracy']
)

In [8]:
model.fit_generator(
    generator = train_generator,
    epochs = 20,
    validation_data = val_generator
)

C:\Users\smhrd\AppData\Local\Temp\ipykernel_17080\647605993.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/20
4/4 [==============================] - 9s 2s/step - loss: 1.6180 - accuracy: 0.4656 - val_loss: 0.7613 - val_accuracy: 0.6750
Epoch 2/20
4/4 [==============================] - 3s 691ms/step - loss: 0.8535 - accuracy: 0.5906 - val_loss: 0.5926 - val_accuracy: 0.7250
Epoch 3/20
4/4 [==============================] - 3s 705ms/step - loss: 0.7172 - accuracy: 0.7219 - val_loss: 0.5347 - val_accuracy: 0.8000
Epoch 4/20
4/4 [==============================] - 3s 681ms/step - loss: 0.5767 - accuracy: 0.7875 - val_loss: 0.4215 - val_accuracy: 0.8750
Epoch 5/20
4/4 [==============================] - 3s 669ms/step - loss: 0.4778 - accuracy: 0.8500 - val_loss: 0.3935 - val_accuracy: 0.8500
Epoch 6/20
4/4 [==============================] - 3s 680ms/step - loss: 0.3950 - accuracy: 0.9125 - val_loss: 0.3970 - val_accuracy: 0.9000
Epoch 7/20
4/4 [==============================] - 3s 905ms/step - loss: 0.3520 - accuracy: 0.8844 - val_loss: 0.2910 - val_accuracy: 0.9000
Epoch 8/20
4/4 [=======

In [9]:
# 저장
from keras.models import load_model
model.save('VGG_model_epoch20.h5')